---
<a href="https://colab.research.google.com/github/hc2x/civl4500/blob/main/sovi_exercise/sovi-assessment-tutorial.ipynb" target="_blank">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a>

# Instruction for Google Colab environment
- In order to make edits to this notebook, you should press File > "Save a Copy in Drive". This will ensure that any edits will be on your local copy, and they will not affect the notebook shared with everyone else.
- Click "Connect" on the top-right corner. Once you see RAM and Disk, you are ready to run the codes!

---

# Introduction
- This Jupyter Notebook demonstrates how to combine socio-economic indicators, flood hazard data, and population data to create a disaster vulnerability assessment at the upazila (sub-district) level in Bangladesh. The steps are as follows:
- All data is from the former study ([Lee et al., 2021](https://nhess.copernicus.org/articles/21/1807/2021/)). It is recommended to overview the paper to understand the concept and specifics of vulnerability indicators.
- Structure of Exercise:

  1. Data Reading and Preprocessing
  2. Min-Max Scaling of Indicators
  3. Weight Calculation of Indicators (Three Methods: Equal Weights, AHP, PCA)
  4. Vulnerability Map Creation
  5. Top 10 Most Vulnerable Upazilas (Bar Charts)
  6. Flood-Hazard-Based Affected Population Estimation
  7. District-Level Population-Weighted Vulnerability

In [ ]:
! pip install rasterio
! pip install shapely

In [ ]:
# Download the ZIP file and save it as exercise-data.zip
!wget -O exercise-data.zip https://github.com/hc2x/civl4500/raw/main/sovi_exercise/exercise-data.zip

# Create the subfolder 'data' if it doesn't already exist
!mkdir -p data

# Unzip the downloaded file into the 'data' folder
!unzip exercise-data.zip -d data

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
import matplotlib as mpl
import matplotlib.pyplot as plt
from rasterio.plot import show
from shapely.geometry import box
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA

# Optional: make plots inline and set figure size
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)

In [ ]:
# Install required packages
# !pip install geopandas rasterio matplotlib scikit-learn shapely

## 1. Data Reading and Preprocessing

### 1.1 Read the vulnerability indicators data
- All data are in `data` folder.
- `data_raw.xlsx` contains 26 socio-economic, health, and adaptive capacity indicators for each upazila (`ADM3_PCODE`).

In [ ]:
# Load data description
description_df = pd.read_excel("./data/data_table.xlsx", index_col=0)
description_df

In [ ]:
# Load data
table_df = pd.read_excel("./data/data_raw.xlsx")
print("Table data shape:")
print(table_df.shape)

# For demonstration, let's just list them:
indicator_cols = [col for col in table_df.columns if col not in ['ADM3_PCODE']]
print("Table data preview:")
display(table_df.head())

In [ ]:
table_df.max()

### 1.2 Read the administrative boundary shapefile

In [ ]:
district = gpd.read_file("./data/bgd_admbnda_adm2_bbs_20180410.gpkg")  # 64 districts
admin_gdf = gpd.read_file("./data/bgd_admbnda_adm3_bbs_20180410.gpkg")  # 544 upazilas
print("Admin boundary preview:")
display(admin_gdf.head())

In [ ]:
# Plot the admin boundaries
fig, ax = plt.subplots(dpi=125)
admin_gdf.plot(ax=ax, linewidth=0.5)
ax.set_title('Admin Boundaries')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
fig.tight_layout()
plt.show()

### 1.3 Read the flood inundation raster
- 1 (Perennial waterbodies), 2 (Flood inundation area)

In [ ]:
# Flood Inundation Area (1: Perennial waterbodies, 2: Flood inundation area)
flood_raster = rasterio.open('./data/bgd_inun_30m.tif') # 1 (Perennial waterbodies), 2 (Flood inundation area)

# Visualize the flood raster
fig, ax = plt.subplots(dpi=125)
show(flood_raster, ax=ax, title='Flood Inundation Area', cmap='Blues')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
fig.tight_layout()
plt.show()

### 1.4 Read the population raster

In [ ]:
from matplotlib.colors import LogNorm

# Open the population raster file.
with rasterio.open('./data/bgd_ppp_2017_30m_decuple.tif') as pop_raster:
    pop_data = pop_raster.read(1)

# Replace non-positive values with NaN to avoid issues with the log scale.
pop_data = np.where(pop_data > 0, pop_data, np.nan)

# Visualize the population data with log scale normalization.
plt.figure(dpi=125)
plt.imshow(pop_data, cmap='Reds', norm=LogNorm())
plt.title('Population Per Pixel (Log-scaled)')
plt.xticks([])
plt.yticks([])
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.colorbar(label='Population')
plt.tight_layout()
plt.show()

## 2. Min-Max Scaling of Indicators
We'll scale each of the `n` indicators from 0 to 1 using min-max normalization.
The formula for min-max scaling:

$$X^\prime = \frac{X - X_{\min}}{X_{\max} - X_{\min}}$$

We'll store these scaled columns in new columns, e.g. col_1_scaled, etc.

In [ ]:
# We'll store these scaled columns in new columns, e.g. col_1_scaled, etc.
scaler = MinMaxScaler()
scaled_values = scaler.fit_transform(table_df[indicator_cols])
scaled_df = pd.DataFrame(scaled_values, columns=indicator_cols)
scaled_df = pd.concat([table_df[['ADM3_PCODE']], scaled_df], axis=1)

print("Scaled table data preview:")
display(scaled_df.head())

In [ ]:
scaled_df.max()

## 3. Weight Calculation of Indicators
We have three different approaches:
1) Equal Weights
2) AHP (Analytic Hierarchy Process) Weights
3) Custom Weights (survey) Weights
4) PCA (Principal Component Analysis)-based Weights

After computing each set of weights, we will store them in a single CSV file (weight_data.csv).

### 3.1 Equal Weights
- Assign the same weight (1/n) to all indicators.

In [ ]:
# If we have n indicators, each weight is 1/n
num_indicators = len(indicator_cols)
equal_weights = np.array([1/num_indicators]*num_indicators)
equal_weights

### 3.2 AHP Weights
- We will construct the pairwise comparison matrix from a small survey among students.
- For demonstration, let's just assume we have a hypothetical matrix. In practice, you will collect pairwise comparison data from the surveyees and construct the matrix.

In [ ]:
# Example: If we have 21 indicators, we have a 21x21 matrix. Here, we create a random example.
# But typically, you'd fill this with real comparisons. Let's just do a small example for demonstration.

# WARNING: The below is a random matrix for demonstration. 
# Real AHP should follow consistent judgments.
#   1) Provide pairwise comparison questions to survey respondents
#   2) Collect their responses (e.g., 1-9 scale)
#   3) Normalize the responses to create a pairwise comparison matrix
#   4) Use the AHP method to derive weights from the pairwise comparison matrix
#   5) Ensure the matrix is consistent (e.g., using the Consistency Ratio)
#   6) Use the derived weights in your analysis

# We'll define a helper function to compute AHP weights from a pairwise matrix.
def ahp_weights(pairwise_matrix):
    # 1. Compute eigenvalues and eigenvectors
    eigenvalues, eigenvectors = np.linalg.eig(pairwise_matrix)
    # 2. Principal eigenvector corresponds to the max eigenvalue
    max_eig_index = np.argmax(eigenvalues)
    principal_eigvec = np.real(eigenvectors[:, max_eig_index])
    # 3. Normalize to get weights
    normalized_weights = principal_eigvec / principal_eigvec.sum()
    return normalized_weights

# For demonstration, let's build a random positive matrix with 21x21
# and enforce diagonal = 1 (because each indicator is equally compared with itself).
rng = np.random.default_rng(42)
pairwise_demo = rng.random((num_indicators, num_indicators))*2 + 0.1  # random [0.1,2.1]
np.fill_diagonal(pairwise_demo, 1.0)

# We need to symmetrize or maintain consistency in a real AHP matrix, but for demonstration:
ahp_weight_vector = ahp_weights(pairwise_demo)

In [ ]:
ahp_weight_vector

### 3.3 Custom weights via Survey
- Survey Form: https://forms.gle/ccJTq3RL4UoQdfoc9
- Survey Result: https://docs.google.com/spreadsheets/d/13WovhBFI_9q1DR-aJv3PBOf5Hlew5ObxEY3R6ICE2xo/edit?usp=sharing

In [ ]:
custom_average = np.array([4.00, 2.00, 5.00, 5.00, 5.00, 5.00, 4.00, 1.00, 1.00, 1.00, 1.00, 4.00, 4.00, 5.00, 5.00, 4.00, 5.00, 4.00, 5.00, 4.00, 5.00, 5.00, 1.00, 5.00, 1.00, 5.00])
custom_weights = custom_average / custom_average.sum()
custom_weights

### 3.4 PCA-based Weights
- We'll use PCA on the scaled socio-economic indicators. 
- One simple approach is to take the loading of the first principal component as weights (or we could combine multiple PCs). 

In [ ]:
scaled_indicator_data = scaled_df[indicator_cols]
pca = PCA(n_components=1)
pca.fit(scaled_indicator_data)

# PCA components_ shape is (n_components, n_features)
# We'll take the absolute value of the loadings, or we can keep the sign. 
# Common approach is to take the absolute values, then normalize to sum = 1
pca_component = pca.components_[0]
pca_abs = np.abs(pca_component)
pca_weights = pca_abs / np.sum(pca_abs)

In [ ]:
# Combine weights into one DataFrame and save
weight_df = pd.DataFrame({
    'Indicator': indicator_cols,
    'Equal_Weight': equal_weights,
    'AHP_Weight': ahp_weight_vector,
    'Custom_Weight': custom_weights,
    'PCA_Weight': pca_weights
})

# Save to CSV
weight_df.to_csv("weight_data.csv", index=False)
print("Weights have been saved to weight_data.csv")
display(weight_df.head())

In [ ]:
# Summation of weights
weight_df[['Equal_Weight','AHP_Weight', 'Custom_Weight', 'PCA_Weight']].sum()

In [ ]:
# Bar plot of weights
plt.figure(dpi=125)
weight_df.set_index('Indicator').plot(kind='bar', alpha=0.7)
plt.title('Weights for Indicators')
plt.xlabel('Indicators')
plt.ylabel('Weights')
plt.xticks(rotation=60)
plt.legend(title='Weight Type')
plt.tight_layout()
plt.show()

## 4. Vulnerability Map Creation

We'll create three vulnerability maps: 
  1) Vulnerability (Equal Weights)
  2) Vulnerability (AHP)
  3) Vulnerability (Custom Weights)
  3) Vulnerability (PCA)

For each approach, 

$$
\text{vulnerability score} = \sum_{i=1}^{n} \left( \text{scaled\_indicator}_{i} \cdot \text{weight}_{i} \right)
$$

In [ ]:
def compute_vulnerability_scores(df, indicator_cols, weight_df, weight_name="Equal_Weights"):
    """
    table_df: must have columns [Upazila_ID, col_1_scaled, ..., col_n_scaled]
    indicator_cols: original list of indicator column names (not scaled)
    weights: 1D array of length n (each indicator's weight)
    """
    scaled_data = df[indicator_cols].values
    weights_selected = weight_df[weight_name].values
    # # Ensure weights are normalized
    # weights_sum = np.sum(weights_selected)
    # if weights_sum != 1:
    #     weights_selected = weights_selected / weights_sum
    # Compute vulnerability score
    vul_scores = np.sum(scaled_data * weights_selected.reshape(1, -1), axis=1)
    return vul_scores

# Compute vulnerability scores using different methods
scaled_df['VUL_EQUAL'] = compute_vulnerability_scores(scaled_df, indicator_cols, weight_df, "Equal_Weight")
scaled_df['VUL_AHP'] = compute_vulnerability_scores(scaled_df, indicator_cols, weight_df, "AHP_Weight")
scaled_df['VUL_CUSTOM'] = compute_vulnerability_scores(scaled_df, indicator_cols, weight_df, "Custom_Weight")
scaled_df['VUL_PCA'] = compute_vulnerability_scores(scaled_df, indicator_cols, weight_df, "PCA_Weight")

print("Vulnerability columns added to scaled_df:")
display(scaled_df.head())

In [ ]:
# Now we can join these vulnerability scores to the admin shapefile for mapping.
scaled_df['ADM3_PCODE'] = scaled_df['ADM3_PCODE'].astype(str)
admin_gdf['ADM3_PCODE'] = admin_gdf['ADM3_PCODE'].astype(str)
# Merge the vulnerability scores with the admin shapefile
# Note: Ensure the join column names are consistent
vul_gdf = admin_gdf.merge(scaled_df, on='ADM3_PCODE', how='left')
vul_gdf

In [ ]:
# Create a figure with constrained_layout enabled
fig, ax = plt.subplots(1, 4, figsize=(12, 6), dpi=125, constrained_layout=True)

# Plot maps without individual legends
vul_gdf.plot(column='VUL_EQUAL', cmap='bwr', legend=False, vmin=0.2, vmax=0.8, ax=ax[0])
ax[0].set_title("Equal Weights")
ax[0].set_xticks([])
ax[0].set_yticks([])
vul_gdf.plot(column='VUL_AHP', cmap='bwr', legend=False, vmin=0.2, vmax=0.8, ax=ax[1])
ax[1].set_title("AHP Weights")
ax[1].set_xticks([])
ax[1].set_yticks([])
vul_gdf.plot(column='VUL_CUSTOM', cmap='bwr', legend=False, vmin=0.2, vmax=0.8, ax=ax[2])
ax[2].set_title("Custom Weights")
ax[2].set_xticks([])
ax[2].set_yticks([])
vul_gdf.plot(column='VUL_PCA', cmap='bwr', legend=False, vmin=0.2, vmax=0.8, ax=ax[3])
ax[3].set_title("PCA Weights")
ax[3].set_xticks([])
ax[3].set_yticks([])

# Plot district boundaries on each axis
district.boundary.plot(ax=ax[0], color='grey', linewidth=0.5)
district.boundary.plot(ax=ax[1], color='grey', linewidth=0.5)
district.boundary.plot(ax=ax[2], color='grey', linewidth=0.5)
district.boundary.plot(ax=ax[3], color='grey', linewidth=0.5)

# Create a common ScalarMappable for the colorbar based on the colormap and normalization
sm = mpl.cm.ScalarMappable(cmap='bwr', norm=mpl.colors.Normalize(vmin=0.2, vmax=0.8))
sm._A = []  # empty array for the ScalarMappable

# Add a single horizontal colorbar below all subplots
cbar = fig.colorbar(sm, ax=ax, orientation='horizontal', fraction=0.05, pad=0.1)
cbar.set_label("Vulnerability Score")
plt.show()